In [1]:
import requests
import pandas as pd
import io

In [2]:
BASE_URL = 'https://www-genesis.destatis.de/genesisWS/rest/2020/'
TOKEN = "5c88a4e4308c45a6880f622053f7c41f" 

headers = {
'Content-Type': 'application/x-www-form-urlencoded',
'username': TOKEN,
'password': ""
}
langPref = "de"

### Test if the connection works

In [3]:
hello = requests.post(BASE_URL + 'helloworld/logincheck',
headers = headers,
data = {
'language': langPref
})

# check the username -> it should not be guest
hello.text[:]

'{"Status":"Sie wurden erfolgreich an- und abgemeldet! Bei mehr als 3 parallelen Requests wurden länger als 15 Minuten laufende Requests beendet.","Username":"5c88a4e4308c45a6880f622053f7c41f"}'

In [ ]:
response = requests.post(BASE_URL + 'data/table',
headers = headers,
data = {
'name': '12411-0005',
'format' : 'json',
'language': 'de'
})
# prüfe, ob die Antwort Text, binär oder leer ist
print(response.text[0:120])



{"Ident":{"Service":"data","Method":"table"},"Status":{"Code":0,"Content":"erfolgreich","Type":"Information"},"Parameter


In [ ]:
data = response.json()
data["Object"]["Content"]

[]


In [ ]:
# csv_text ist dein Output aus data["Object"]["Content"]
csv_text = data["Object"]["Content"]

# Die CSV hat ein paar einleitende Zeilen (Tabelle:..., Bevölkerung:...), wir überspringen sie
# In diesem Beispiel skiprows=5 funktioniert meistens gut
df = pd.read_csv(io.StringIO(csv_text), sep=';', skiprows=6, engine='python')

# Spalten umbenennen: erste Spalte enthält Altersgruppen
df.rename(columns={df.columns[0]: 'Altersgruppe'}, inplace=True)

# Optional: alle Werte in int umwandeln
for col in df.columns[1:]:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Ergebnis prüfen
print(df.head())

   Altersgruppe  31.12.2020  31.12.2021  31.12.2022  31.12.2023  31.12.2024
0  unter 1 Jahr    769380.0    791254.0    724711.0    689834.0    673380.0
1     1-Jährige    783593.0    780795.0    786020.0    739396.0    700940.0
2     2-Jährige    798366.0    789145.0    773830.0    793110.0    744162.0
3     3-Jährige    806198.0    803334.0    782423.0    780001.0    797030.0
4     4-Jährige    811601.0    810805.0    797764.0    789018.0    783616.0


In [78]:
df = df.dropna()

In [79]:
df

,Altersgruppe,31.12.2020,31.12.2021,31.12.2022,31.12.2023,31.12.2024
0,unter 1 Jahr,769380.0,791254.0,724711.0,689834.0,673380.0
1,1-Jährige,783593.0,780795.0,786020.0,739396.0,700940.0
2,2-Jährige,798366.0,789145.0,773830.0,793110.0,744162.0
3,3-Jährige,806198.0,803334.0,782423.0,780001.0,797030.0
4,4-Jährige,811601.0,810805.0,797764.0,789018.0,783616.0
...,...,...,...,...,...,...
82,82-Jährige,687458.0,732760.0,722205.0,681158.0,553001.0
83,83-Jährige,604464.0,644569.0,675973.0,678282.0,641590.0
84,84-Jährige,544669.0,561973.0,589512.0,630184.0,633529.0
85,85 Jahre und mehr,2505932.0,2625444.0,2699436.0,2844213.0,3025803.0


### Anpassung Bundesland

In [130]:
BASE_URL = 'https://www-genesis.destatis.de/genesisWS/rest/2020/'
TOKEN = "5c88a4e4308c45a6880f622053f7c41f" 

headers = {
'Content-Type': 'application/x-www-form-urlencoded',
'username': TOKEN,
'password': ""
}
langPref = "de"

In [131]:
response = requests.post(BASE_URL + 'data/table',
headers = headers,
data = {
'name': '12411-0010',
'format' : 'json',
'language': 'de',
"regionalvariable": "GEMEINDE",
"regionalkey": "06412000",
"startyear": "2010",
"endyear": "2024"
})
# prüfe, ob die Antwort Text, binär oder leer ist
print(response.text[0:120])



{"Ident":{"Service":"data","Method":"table"},"Status":{"Code":22,"Content":"erfolgreich (Mindestens ein Parameter enthäl


In [132]:
data = response.json()
data["Object"]["Content"]

'Tabelle: 12411-0010\nBevölkerung: Bundesländer, Stichtag;;;;;;;;;;;;;;;\nFortschreibung des Bevölkerungsstandes;;;;;;;;;;;;;;;\nBevölkerungsstand (Anzahl);;;;;;;;;;;;;;;\n;Stichtag;Stichtag;Stichtag;Stichtag;Stichtag;Stichtag;Stichtag;Stichtag;Stichtag;Stichtag;Stichtag;Stichtag;Stichtag;Stichtag;Stichtag\n;31.12.2010;31.12.2011;31.12.2012;31.12.2013;31.12.2014;31.12.2015;31.12.2016;31.12.2017;31.12.2018;31.12.2019;31.12.2020;31.12.2021;31.12.2022;31.12.2023;31.12.2024\nBaden-Württemberg;10753880;10512441;10569111;10631278;10716644;10879618;10951893;11023425;11069533;11100394;11103043;11124642;11167721;11230740;11245898\nBayern;12538696;12443372;12519571;12604244;12691568;12843514;12930751;12997204;13076721;13124737;13140183;13176989;13105221;13176426;13248928\nBerlin;3460725;3326002;3375222;3421829;3469849;3520031;3574830;3613495;3644826;3669491;3664088;3677472;3632853;3662381;3685265\nBrandenburg;2503273;2453180;2449511;2449193;2457872;2484826;2494648;2504040;2511917;2521893;2531071

In [133]:
# csv_text ist dein Output aus data["Object"]["Content"]
csv_text = data["Object"]["Content"]

df = pd.read_csv(io.StringIO(csv_text), sep=';', skiprows=5, engine='python')

# Alle Spalten außer der ersten (Bundesland) transformieren
df.columns = [col[-4:] if col.startswith("31.12.") else "Bundesland" for col in df.columns]
df = df.dropna()

df = df.melt(id_vars="Bundesland", var_name="Jahr",value_name="Bevölkerung")

In [134]:
# Input variable 
bl = "Sachsen"

In [135]:
df1 = df[df["Bundesland"] == bl]

df1["Bevölkerung_last_year"] = df1["Bevölkerung"].shift(1)
df1["Bevölkerung_5_year"] = df1["Bevölkerung"].shift(5)

# percentage development
df1["Perc_Change1"] = (df1["Bevölkerung"] / df1["Bevölkerung_last_year"] -1)*100
df1["Perc_Change5"] = (df1["Bevölkerung"] / df1["Bevölkerung_5_year"] -1)*100

df1["Perc_Change1"] = df1["Perc_Change1"].round(2)
df1["Perc_Change5"] = df1["Perc_Change5"].round(2)

/var/folders/4c/4pm4bwz11292qrczgwq1dmfw0000gn/T/ipykernel_2238/2689466114.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["Bevölkerung_last_year"] = df1["Bevölkerung"].shift(1)
/var/folders/4c/4pm4bwz11292qrczgwq1dmfw0000gn/T/ipykernel_2238/2689466114.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["Bevölkerung_5_year"] = df1["Bevölkerung"].shift(5)
/var/folders/4c/4pm4bwz11292qrczgwq1dmfw0000gn/T/ipykernel_2238/2689466114.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy

### Arbeitslosenquote

In [137]:
BASE_URL = 'https://www-genesis.destatis.de/genesisWS/rest/2020/'
TOKEN = "5c88a4e4308c45a6880f622053f7c41f" 

headers = {
'Content-Type': 'application/x-www-form-urlencoded',
'username': TOKEN,
'password': ""
}
langPref = "de"

In [167]:
response = requests.post(BASE_URL + 'data/table',
headers = headers,
data = {
'name': '13211-0007',
'format' : 'json',
'language': 'de',
"startyear": "2010",
"endyear": "2024"
})
# prüfe, ob die Antwort Text, binär oder leer ist
print(response.text[0:120])



{"Ident":{"Service":"data","Method":"table"},"Status":{"Code":0,"Content":"erfolgreich","Type":"Information"},"Parameter


In [174]:
data = response.json()
data["Object"]["Content"]

'Tabelle: 13211-0007\nArbeitslose, Arbeitslosenquoten, Gemeldete Arbeitsstellen:;;;;;\nBundesländer, Jahre;;;;;\nArbeitsmarktstatistik der Bundesagentur für Arbeit;;;;;\n;;Arbeitslose;Arbeitslosenquote aller zivilen Erwerbspersonen;Arbeitslosenquote d. abhängigen ziv. Erwerbspers.;Gemeldete Arbeitsstellen\n;;Anzahl;Prozent;Prozent;Anzahl\nBaden-Württemberg;2010;272656;4,9;5,4;50969\nBaden-Württemberg;2011;226918;4,0;4,5;73278\nBaden-Württemberg;2012;222202;3,9;4,4;74844\nBaden-Württemberg;2013;233956;4,1;4,5;63506\nBaden-Württemberg;2014;230372;4,0;4,4;71974\nBaden-Württemberg;2015;227098;3,8;4,3;82009\nBaden-Württemberg;2016;226421;3,8;4,2;92333\nBaden-Württemberg;2017;212837;3,5;3,9;102096\nBaden-Württemberg;2018;195128;3,2;3,5;112360\nBaden-Württemberg;2019;196950;3,2;3,5;106185\nBaden-Württemberg;2020;259940;4,1;4,5;72963\nBaden-Württemberg;2021;247774;3,9;4,3;88314\nBaden-Württemberg;2022;223119;3,5;3,9;111188\nBaden-Württemberg;2023;245466;3,9;4,2;95715\nBaden-Württemberg;2024;26

In [175]:
# csv_text ist dein Output aus data["Object"]["Content"]
csv_text = data["Object"]["Content"]
df_al = pd.read_csv(io.StringIO(csv_text), sep=';', skiprows=5, engine='python')

In [176]:
# Alle Spalten außer der ersten (Bundesland) transformieren
df_al = df_al.iloc[:,:-3]
df_al.columns = ["Bundesland", "Jahr", "Arbeitslose"]
df_al = df_al.dropna()

df_al["Jahr"] = df_al["Jahr"].round()

df_al["Jahr"] = df_al["Jahr"].astype("str").str[:4]
df_al = df_al[df_al["Bundesland"] == bl]

### Merge

In [181]:
df_merge = df1.merge(df_al, how="left", on = ["Bundesland", "Jahr"])

In [182]:
df_merge

,Bundesland,Jahr,Bevölkerung,Bevölkerung_last_year,Bevölkerung_5_year,Perc_Change1,Perc_Change5,Arbeitslose
0,Sachsen,2010,4149477.0,NaN,NaN,NaN,NaN,253198.0
1,Sachsen,2011,4054182.0,4149477.0,NaN,-2.30,NaN,226887.0
2,Sachsen,2012,4050204.0,4054182.0,NaN,-0.10,NaN,207826.0
3,Sachsen,2013,4046385.0,4050204.0,NaN,-0.09,NaN,200628.0
4,Sachsen,2014,4055274.0,4046385.0,NaN,0.22,NaN,187494.0
5,Sachsen,2015,4084851.0,4055274.0,4149477.0,0.73,-1.56,174319.0
6,Sachsen,2016,4081783.0,4084851.0,4054182.0,-0.08,0.68,157862.0
7,Sachsen,2017,4081308.0,4081783.0,4050204.0,-0.01,0.77,140348.0
8,Sachsen,2018,4077937.0,4081308.0,4046385.0,-0.08,0.78,126311.0
9,Sachsen,2019,4071971.0,4077937.0,4055274.0,-0.15,0.41,116051.0


### ÖPNV

In [208]:
input_adresse = "Meckenheimer Straße 18B, 67454 Haßloch"

In [209]:
import requests

def geocode(address):
    url = "https://nominatim.openstreetmap.org/search"
    params = {
        'q': address,
        'format': 'json',
        'addressdetails': 1,
        'limit': 1
    }
    headers = {
        'User-Agent': 'ImmobilienStandortScore/1.0'
    }

    r = requests.get(url, params=params, headers=headers)
    data = r.json()

    if len(data) == 0:
        return None
    
    return float(data[0]['lat']), float(data[0]['lon'])

# Beispiel
lat, lon = geocode(input_adresse)
print(lat, lon)

49.373432 8.2535726


In [210]:
lat

49.373432

In [211]:
def get_oepnv_dichte(lat, lon, radius=800):
    query = f"""
    [out:json][timeout:25];
    (
      node["public_transport"="platform"](around:{radius},{lat},{lon});
      node["railway"="station"](around:{radius},{lat},{lon});
      node["railway"="halt"](around:{radius},{lat},{lon});
      node["railway"="tram_stop"](around:{radius},{lat},{lon});
      node["railway"="subway_entrance"](around:{radius},{lat},{lon});
      node["highway"="bus_stop"](around:{radius},{lat},{lon});
    );
    out body;
    """
    
    response = requests.post("https://overpass-api.de/api/interpreter", data=query)
    data = response.json()

    elements = []
    for element in data['elements']:
        elements.append({
            "type": element.get("tags", {}).get("public_transport") or
                    element.get("tags", {}).get("railway") or
                    element.get("tags", {}).get("highway"),
            "lat": element["lat"],
            "lon": element["lon"]
        })
        
    df = pd.DataFrame(elements)
    return df


stations = get_oepnv_dichte(lat, lon)
print(stations)
print("Anzahl Stationen:", len(stations))

       type        lat       lon
0   station  49.371900  8.252504
1  platform  49.372309  8.263223
2  platform  49.371049  8.260596
3  platform  49.372307  8.253806
4  platform  49.368251  8.257849
5  platform  49.368160  8.257995
6  platform  49.369915  8.250787
7  platform  49.371933  8.254901
8  platform  49.369867  8.251323
9  platform  49.371911  8.255228
Anzahl Stationen: 10


In [212]:
len(stations[stations['type'] == 'station'])

1

In [184]:
anzahl_station = len(stations)

bus = len(stations[stations['type'] == 'bus_stop'])
tram = len(stations[stations['type'] == 'tram_stop'])
bahn = len(stations[stations['type'] == 'station'])
subway = len(stations[stations['type'] == 'subway_entrance'])

In [185]:
import math

def station_density(count, radius):
    area_km2 = math.pi * (radius/1000)**2
    return count / area_km2

density = station_density(len(stations), 800)
print("Stationsdichte:", density)

Stationsdichte: 41.778172561622526


### Supermärkte

In [218]:
def get_supermarkets(lat, lon, radius=800):
    query = f"""
    [out:json][timeout:25];
    (
      node["shop"="supermarket"](around:{radius},{lat},{lon});
      node["shop"="grocery"](around:{radius},{lat},{lon});
    );
    out body;
    """
    
    r = requests.post("https://overpass-api.de/api/interpreter", data=query)
    data = r.json()

    elements = []
    for element in data['elements']:
        elements.append({
            "name": element.get("tags", {}).get("name", "Unknown"),
            "lat": element["lat"],
            "lon": element["lon"],
            "type": element.get("tags", {}).get("shop")
        })
    
    df = pd.DataFrame(elements)
    return df

# Beispiel: München Marienplatz
df = get_supermarkets(lat, lon, 2000)
print(df)
print(f"Anzahl Supermärkte: {len(df)}")

             name        lat       lon         type
0        ALDI Süd  49.365804  8.247450  supermarket
1  EDEKA Stiegler  49.367428  8.266383  supermarket
2           PENNY  49.367149  8.265148  supermarket
3            REWE  49.356419  8.260573  supermarket
Anzahl Supermärkte: 4


In [220]:
lon

8.2535726